# CellViT Classification Dataset — Patch and Label Generation

**Purpose:**  
This notebook prepares **256x256 classification patches** specifically for  
the *CellViT classification head*. Each patch contains cropped H&E tissue  
and a corresponding label file listing nucleus centers and classes.

---

### What It Does
1. Loads processed maps (`inst_map`, `type_map`) from `whole_slide/`.
2. Locates the corresponding RGB `.svs` or `.tif` image.
3. Tiles the slide into non-overlapping 256x256 patches.
4. For each patch:
   - Saves an RGB patch (`.png`)
   - Extracts nucleus centroids and class IDs → saves as `.csv`
5. Produces a clean folder structure compatible with CellViT’s training pipeline.
6. Generate `inst_map` & `type_map` for 256x256 patches

---

### Output Folder Structure
ProcessedDataset/

└── v1_40x_area20/

└── patches_cellvit_p256/

├── images/ ← 256x256 cropped RGB patches

├── labels/ ← matching .csv files with x,y,class

├── debug/ ← visualization or QA exports

└── _meta/ ← configuration and logs


---

### Table of Contents
- [Step 1 — Setup and Folder Creation](#step1)
- [Step 2 — Patch Extraction and Label Generation](#step2)
- [Step 3 — Visual Inspection of Generated Patches](#step3)
- [Step 4 — Summary and Quality Checks](#step4)
- [Step 5 — Dataset Integrity + Overview](#step5)
- [Step 6 — Crop and save 256x256 patch-level inst/type maps](#step6)



<a id="step1"></a>
# Step 1 — Setup and Folder Creation


In [2]:
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Config ---
PROJECT_ROOT = Path("/projectnb/ec500kb/projects/Fall_2025_Projects/Project_2/AI-guided-whole-slide-imaging-analysis")
PROC_ROOT = PROJECT_ROOT / "ProcessedDataset/v1_40x_area20"
PATCH_SIZE = 256
TASK_NAME = f"patches_cellvit_p{PATCH_SIZE}"

# --- Input directories ---
RAW_IMG_ROOT = PROJECT_ROOT / "Datasets/BU_course_data/TrainingData/TrainingImages_and_Annotations"
INST_DIR = PROC_ROOT / "whole_slide/inst_maps"
TYPE_DIR = PROC_ROOT / "whole_slide/type_maps"

# --- Output directories ---
PATCH_ROOT = PROC_ROOT / TASK_NAME
(PATCH_ROOT / "images").mkdir(parents=True, exist_ok=True)
(PATCH_ROOT / "labels").mkdir(parents=True, exist_ok=True)
(PATCH_ROOT / "debug").mkdir(parents=True, exist_ok=True)
(PATCH_ROOT / "_meta").mkdir(parents=True, exist_ok=True)

# --- Metadata loading ---
with open(PROC_ROOT / "_meta/class_map.json", "r") as f:
    class_map = json.load(f)

# --- Record run parameters ---
params = {
    "task": "cellvit_classification_head",
    "patch_size": PATCH_SIZE,
    "input_root": str(PROC_ROOT),
    "output_root": str(PATCH_ROOT),
    "num_classes": len(class_map),
    "class_map": class_map,
}

with open(PATCH_ROOT / "_meta/params.json", "w") as f:
    json.dump(params, f, indent=2)

# --- Confirmation ---
print(f"✅ Initialized output structure for CellViT classification patches:")
print(f"  → {PATCH_ROOT}")
print(f"  Classes: {list(class_map.values())}")


✅ Initialized output structure for CellViT classification patches:
  → /projectnb/ec500kb/projects/Fall_2025_Projects/Project_2/AI-guided-whole-slide-imaging-analysis/ProcessedDataset/v1_40x_area20/patches_cellvit_p256
  Classes: ['background', 'epithelial', 'lymphocyte', 'macrophage', 'neutrophil']


<a id="step2"></a>
# Step 2 — Patch Extraction and Label Generation

In [3]:
from skimage.measure import regionprops
import cv2
from openslide import OpenSlide

def find_image_path(slide_name: str) -> Path | None:
    """
    Locate the corresponding .svs (preferred) or .tif file for a given slide.
    """
    for patient_dir in sorted(RAW_IMG_ROOT.iterdir()):
        if not patient_dir.is_dir():
            continue
        for ext in (".svs", ".tif"):
            candidate = patient_dir / f"{slide_name}{ext}"
            if candidate.exists():
                return candidate
    return None


def load_rgb_image(img_path: Path) -> np.ndarray:
    """
    Load RGB image using OpenSlide (.svs) or OpenCV (.tif).
    """
    if img_path.suffix.lower() == ".svs":
        with OpenSlide(str(img_path)) as slide:
            w, h = slide.level_dimensions[0]
            rgb = np.array(slide.read_region((0, 0), 0, (w, h)))[:, :, :3]
        return rgb
    else:
        img = cv2.imread(str(img_path))
        if img is None:
            raise RuntimeError(f"Failed to load {img_path}")
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def extract_patch_labels(inst_map: np.ndarray, type_map: np.ndarray):
    """
    Return (x, y, class_id) for each nucleus centroid.
    """
    rows = []
    for region in regionprops(inst_map):
        y, x = region.centroid
        cls_vals = type_map[inst_map == region.label]
        if cls_vals.size == 0:
            continue
        cls = int(np.bincount(cls_vals).argmax())  # majority vote
        if cls == 0:
            continue  # skip background
        rows.append((int(x), int(y), cls))
    return rows


# --- Main loop ---
slides = sorted([p.stem.replace("_inst", "") for p in INST_DIR.glob("*_inst.npy")])
print(f"Found {len(slides)} processed slides.")

for slide_name in tqdm(slides, desc="Generating patches"):
    inst_path = INST_DIR / f"{slide_name}_inst.npy"
    type_path = TYPE_DIR / f"{slide_name}_type.npy"

    if not inst_path.exists() or not type_path.exists():
        print(f"⚠️ Missing maps for {slide_name}")
        continue

    try:
        inst_map = np.load(inst_path)
        type_map = np.load(type_path)
        img_path = find_image_path(slide_name)
        if img_path is None:
            print(f"⚠️ No image found for {slide_name}")
            continue
        rgb = load_rgb_image(img_path)

        h, w = rgb.shape[:2]
        saved_count = 0

        for y in range(0, h, PATCH_SIZE):
            for x in range(0, w, PATCH_SIZE):
                patch_rgb = rgb[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                patch_inst = inst_map[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
                patch_type = type_map[y:y+PATCH_SIZE, x:x+PATCH_SIZE]

                if patch_rgb.shape[0] != PATCH_SIZE or patch_rgb.shape[1] != PATCH_SIZE:
                    continue  # skip incomplete edge tiles

                # skip empty patches
                if np.count_nonzero(patch_inst) == 0:
                    continue

                # extract centers
                centers = extract_patch_labels(patch_inst, patch_type)
                if not centers:
                    continue

                # save patch
                img_name = f"{slide_name}_{x}_{y}.png"
                csv_name = f"{slide_name}_{x}_{y}.csv"

                cv2.imwrite(str(PATCH_ROOT / "images" / img_name), cv2.cvtColor(patch_rgb, cv2.COLOR_RGB2BGR))
                pd.DataFrame(centers, columns=["x", "y", "class"]).to_csv(
                    PATCH_ROOT / "labels" / csv_name, index=False, header=False
                )
                saved_count += 1

        print(f"Saved {saved_count} patches from {slide_name}")

    except Exception as e:
        print(f"⚠️ Error in {slide_name}: {e}")

print("✅ Patch generation complete.")


Found 209 processed slides.


Generating patches:   0%|          | 1/209 [00:00<02:46,  1.25it/s]

Saved 16 patches from TCGA-55-1594-01Z-00-DX1_001


Generating patches:   1%|▏         | 3/209 [00:01<01:08,  2.99it/s]

Saved 4 patches from TCGA-55-1594-01Z-00-DX1_002
Saved 4 patches from TCGA-55-1594-01Z-00-DX1_003


...

Saved 1 patches from TCGA-X4-A8KQ-01Z-00-DX8-1
Saved 1 patches from TCGA-X4-A8KQ-01Z-00-DX8-2
Saved 0 patches from TCGA-X4-A8KQ-01Z-00-DX8-3
Saved 16 patches from TCGA-YL-A9WY-01Z-00-DX1-1


Generating patches:  99%|█████████▉| 207/209 [00:42<00:00,  6.73it/s]

Saved 9 patches from TCGA-YL-A9WY-01Z-00-DX1-2
Saved 1 patches from TCGA-YL-A9WY-01Z-00-DX1-3


Generating patches: 100%|██████████| 209/209 [00:43<00:00,  4.85it/s]

Saved 1 patches from TCGA-YL-A9WY-01Z-00-DX1-4
Saved 0 patches from TCGA-YL-A9WY-01Z-00-DX1-5
✅ Patch generation complete.


<a id="step3"></a>
# Step 3 — Visual Inspection of Generated Patches


In [ ]:
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from matplotlib.patches import Patch
import matplotlib

get_cmap = matplotlib.colormaps.get_cmap
cmap = get_cmap("Set2")
num_classes = len(class_map)
LUT = cmap(np.linspace(0, 1, num_classes))[:, :3]  # RGB LUT in [0,1]


patches = sorted(os.listdir(PATCH_ROOT / "images"))
print(f"Available patches: {len(patches)}")

def visualize_patch_with_gt(patch_name: str):
    if not patch_name:
        return
    clear_output(wait=True)
    print(f"Visualizing patch: {patch_name}")

    base_name = Path(patch_name).stem
    img_path  = PATCH_ROOT / "images" / f"{base_name}.png"
    csv_path  = PATCH_ROOT / "labels" / f"{base_name}.csv"

    parts = base_name.split("_")
    slide_name = "_".join(parts[:-2])
    x_off, y_off = map(int, parts[-2:])
    type_path = TYPE_DIR / f"{slide_name}_type.npy"

    if not type_path.exists() or not img_path.exists() or not csv_path.exists():
        print(f"Missing one of: {type_path.name}, {img_path.name}, {csv_path.name}")
        return

    type_map = np.load(type_path)
    type_patch = type_map[y_off:y_off+PATCH_SIZE, x_off:x_off+PATCH_SIZE]
    img = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
    df = pd.read_csv(csv_path, header=None, names=["x", "y", "class"])

    overlay = img.copy()
    for cls_id, grp in df.groupby("class"):
        color = (np.array(LUT[int(cls_id) % num_classes]) * 255).astype(int)
        for _, row in grp.iterrows():
            center = (int(row.x), int(row.y))
            cv2.circle(overlay, center, 4, (0, 0, 0), -1, lineType=cv2.LINE_AA)
            cv2.circle(overlay, center, 2, color.tolist(), -1, lineType=cv2.LINE_AA)

    tm_clipped = np.clip(type_patch, 0, num_classes - 1).astype(int)
    tm_color = (LUT[tm_clipped] * 255).astype(np.uint8)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img); axes[0].set_title("H&E Patch"); axes[0].axis("off")
    axes[1].imshow(tm_color); axes[1].set_title("Ground Truth Type Map"); axes[1].axis("off")
    axes[2].imshow(overlay); axes[2].set_title("Overlay (Labels on RGB)"); axes[2].axis("off")
    plt.tight_layout(); plt.show()

    present = sorted(df["class"].unique())
    legend_patches = [
        Patch(color=LUT[int(c)], label=f"{int(c)}: {class_map[str(int(c))]}")
        for c in present
    ]
    plt.figure(figsize=(6, 0.6 + 0.3 * len(present)))
    plt.legend(handles=legend_patches, loc="center left",
               bbox_to_anchor=(0, 0.5), frameon=False, title="Cell Types")
    plt.axis("off")
    plt.show()

# --- Proper interactive output setup ---
dropdown = widgets.Dropdown(options=patches, description="Select Patch:")
out = widgets.interactive_output(visualize_patch_with_gt, {'patch_name': dropdown})

display(dropdown, out)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 139.3 MB/s eta 0:00:00


Matplotlib is building the font cache; this may take a moment.


Available patches: 1106


<a id="step4"></a>
# Step 4 — Dataset Summary & Integrity Checks

In [ ]:
from collections import Counter
import seaborn as sns

patch_dir = PATCH_ROOT / "images"
label_dir = PATCH_ROOT / "labels"
meta_dir  = PROC_ROOT / "_meta"
meta_dir.mkdir(exist_ok=True)

records = []
class_counter = Counter()

for csv_file in sorted(label_dir.glob("*.csv")):
    base = csv_file.stem
    img_path = patch_dir / f"{base}.png"
    if not img_path.exists():
        continue

    df = pd.read_csv(csv_file, header=None, names=["x", "y", "class"])
    num_cells = len(df)
    class_counts = df["class"].value_counts().to_dict()

    for cls, n in class_counts.items():
        class_counter[int(cls)] += n

    records.append({
        "patch": base,
        "num_cells": num_cells,
        "unique_classes": list(class_counts.keys()),
        "empty": num_cells == 0
    })

summary_df = pd.DataFrame(records)
summary_df.to_csv(meta_dir / "dataset_summary.csv", index=False)
print(f"Saved summary: {meta_dir / 'dataset_summary.csv'}")

# --- Overview stats ---
total_patches = len(summary_df)
empty_patches = summary_df["empty"].sum()
print(f"Total patches: {total_patches}")
print(f"Empty patches: {empty_patches} ({empty_patches/total_patches:.1%})")

# --- Class distribution ---
class_df = pd.DataFrame.from_dict(class_counter, orient="index", columns=["count"])
class_df["label"] = class_df.index.map(lambda i: class_map.get(str(i), "unknown"))
class_df = class_df.sort_index()

plt.figure(figsize=(7,4))
sns.barplot(x="label", y="count", data=class_df, palette="tab10")
plt.title("Cell Counts per Class")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Cells per patch distribution ---
plt.figure(figsize=(6,4))
sns.histplot(summary_df["num_cells"], bins=40, kde=False)
plt.title("Distribution of Cell Counts per Patch")
plt.xlabel("Number of cells per patch")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


<a id="step5"></a>
# Step 5: Dataset Integrity + Overview

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np

print("🔍 Checking dataset integrity...")

images_dir = PATCH_ROOT / "images"
labels_dir = PATCH_ROOT / "labels"

image_files = sorted(f.stem for f in images_dir.glob("*.png"))
label_files = sorted(f.stem for f in labels_dir.glob("*.csv"))

# 1️⃣  File pairing
unmatched_images = set(image_files) - set(label_files)
unmatched_labels = set(label_files) - set(image_files)

print(f"Total images: {len(image_files)}")
print(f"Total labels: {len(label_files)}")
if unmatched_images:
    print(f"⚠️ {len(unmatched_images)} images without labels.")
if unmatched_labels:
    print(f"⚠️ {len(unmatched_labels)} labels without images.")

# 2️⃣  Validate CSV content (quick sample)
empty_labels = []
class_counter = Counter()
for csv_path in labels_dir.glob("*.csv"):
    try:
        data = np.loadtxt(csv_path, delimiter=",")
        if data.ndim == 1 and data.size == 0:
            empty_labels.append(csv_path.name)
        else:
            # 3rd column = class id
            cls = data[:, 2].astype(int) if data.ndim > 1 else []
            class_counter.update(cls.tolist())
    except Exception:
        empty_labels.append(csv_path.name)

print(f"Empty label files: {len(empty_labels)}")
if empty_labels:
    print("Example empty files:", empty_labels[:3])

# 3️⃣  Patch count per patient
# Ensure file stems are strings
file_series = pd.Series([str(f) for f in image_files])

patient_counts = (
    file_series
    .str.extract(r"(TCGA-[0-9A-Z-]+)")
    .dropna()[0]
    .value_counts()
)

print("\nPatch count per patient (top 5):")
print(patient_counts.head())


# 4️⃣  Class totals
print("\nClass pixel count summary:")
for cid, count in class_counter.items():
    name = class_map.get(str(cid), f"id_{cid}")
    print(f"  {cid} ({name}): {count}")

# 5️⃣  Save summary CSV
summary_path = PROC_ROOT / "_meta" / "final_summary.csv"
pd.DataFrame.from_dict(class_counter, orient="index", columns=["count"]).to_csv(summary_path)
print(f"\n✅ Final summary saved to: {summary_path}")


<a id="step6"></a>
# Step 6: Generate 256x256 patch-level instance and type maps

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import random

print("\nCropping 256x256 instance and type maps...")

from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Config ---
PROJECT_ROOT = Path("/projectnb/ec500kb/projects/Fall_2025_Projects/Project_2/AI-guided-whole-slide-imaging-analysis")
PROC_ROOT = PROJECT_ROOT / "ProcessedDataset/v1_40x_area20"
PATCH_SIZE = 256
TASK_NAME = f"patches_cellvit_p{PATCH_SIZE}"

# --- Input directories ---
RAW_IMG_ROOT = PROJECT_ROOT / "Datasets/BU_course_data/TrainingData/TrainingImages_and_Annotations"
INST_DIR = PROC_ROOT / "whole_slide/inst_maps"
TYPE_DIR = PROC_ROOT / "whole_slide/type_maps"

# --- Output directories ---
PATCH_ROOT = PROC_ROOT / TASK_NAME

PATCH_SIZE = 256
patch_map_dir = PATCH_ROOT / "patch_label_maps"
patch_map_dir.mkdir(exist_ok=True)

slide_summaries = []

for inst_path in INST_DIR.glob("*_inst.npy"):
    slide_id = inst_path.stem.replace("_inst", "")
    type_path = TYPE_DIR / f"{slide_id}_type.npy"
    if not type_path.exists():
        print(f"⚠️ Missing type map for {slide_id}")
        continue

    inst_map = np.load(inst_path)
    type_map = np.load(type_path)
    h, w = inst_map.shape
    saved = 0

    for y in range(0, h, PATCH_SIZE):
        for x in range(0, w, PATCH_SIZE):
            patch_inst = inst_map[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            patch_type = type_map[y:y+PATCH_SIZE, x:x+PATCH_SIZE]
            if patch_inst.shape != (PATCH_SIZE, PATCH_SIZE):
                continue
            if np.all(patch_type == 0):
                continue
            np.save(
                patch_map_dir / f"{slide_id}_{x}_{y}.npy",
                {"inst_map": patch_inst, "type_map": patch_type}
            )
            saved += 1

    slide_summaries.append({"slide": slide_id, "patch_maps": saved})
    print(f"{slide_id}: saved {saved} patch label maps")


In [ ]:
# --- Summary ---
import pandas as pd
summary_df = pd.DataFrame(slide_summaries)
summary_df.to_csv(PROC_ROOT / "_meta" / "patch_label_summary.csv", index=False)
print(f"\n✅ Patch-level maps saved to: {patch_map_dir}")
print(f"Summary written to: _meta/patch_label_summary.csv")

# --- Quick visual sanity check ---
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

# pick a random patch label map
sample = random.choice(list(patch_map_dir.glob("*.npy")))
data = np.load(sample, allow_pickle=True).item()
inst, typ = data["inst_map"], data["type_map"]

# infer corresponding RGB image file
# e.g. slide_id_x_y.npy → slide_id_x_y.png
img_name = sample.stem + ".png"
img_path = PATCH_ROOT / "images" / img_name

rgb = None
if img_path.exists():
    rgb = cv2.imread(str(img_path))
    rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
else:
    print(f"⚠️ No matching RGB image found for {img_name}")

plt.figure(figsize=(15,5))
if rgb is not None:
    plt.subplot(1,3,1); plt.imshow(rgb); plt.title("Original RGB"); plt.axis("off")

plt.subplot(1,3,2); plt.imshow(inst, cmap="nipy_spectral"); plt.title("Instance Map"); plt.axis("off")
plt.subplot(1,3,3); plt.imshow(typ, cmap="tab10"); plt.title("Type Map"); plt.axis("off")
plt.suptitle(sample.stem)
plt.tight_layout()
plt.show()
